In [2]:
import pandas as pd
import numpy as np

In [ ]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])


In [5]:
# preparation

bdf = pd.read_csv('../simulations/large_test/outcomes.csv')

idx_to_artid = np.unique(bdf[['winner', 'loser']].values.ravel())
artid_to_idx = { artid: i for i, artid in enumerate(idx_to_artid) }

art_properties = pd.read_csv('../simulations/large_test/artworks.csv').set_index('id')

art_properties['model_flag'] = art_properties['creator'].case_when(
    caselist=[
        (art_properties.creator == art_properties.artist, 'human'),
        (np.full_like(art_properties.creator, True, dtype=bool), art_properties.creator)
    ]
)

idx_to_is_ai = np.array([art_properties.loc[el, 'is_ai'] for el in idx_to_artid])

idx_to_model = np.array([art_properties.loc[el, 'model_flag'] for el in idx_to_artid])

idx_to_model

def _custom_key(el):
    if el == 'human':
        return 0
    else:
        return 1

model_encoding = { model : i for i, model in enumerate(sorted(np.unique(idx_to_model), key = _custom_key ))} # make human 0

winners = bdf['winner'].map(lambda el: artid_to_idx[el])
losers = bdf['loser'].map(lambda el: artid_to_idx[el])

winners_model = winners.map(lambda el: model_encoding[idx_to_model[el]])
losers_model = losers.map(lambda el: model_encoding[idx_to_model[el]])

reasons = bdf['reason']

print(reasons)



0      5
1      4
2      2
3      0
4      2
      ..
315    0
316    0
317    0
318    0
319    0
Name: reason, Length: 320, dtype: int64


In [ ]:
winners_model_one_hot = get_one_hot(winners_model, len(np.unique(winners_model)))
losers_model_one_hot = get_one_hot(losers_model, len(np.unique(losers_model)))
diff_model_one_hot = winners_model_one_hot - losers_model_one_hot

diff_model_one_hot = diff_model_one_hot[:, 1:]
print(diff_model_one_hot.shape)

reasons_one_hot = get_one_hot(reasons.values, 6)
reasons_one_hot